<a href="https://colab.research.google.com/github/abdulrahmanalaa123/Graduation_Project/blob/Honda/grad_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

imports


In [1]:
!pip3 install pickle5
import pickle5 as pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 7.4 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp38-cp38-linux_x86_64.whl size=236295 sha256=476818795c0234ef812836976006d4399581c30ca85a28fbd7bca8df7a97f519
  Stored in directory: /root/.cache/pip/wheels/25/d4/61/dbd8edd1a0d656be7b4267c85db3b61951eb60016a0154a122
Successfully built pickle5


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from re import L
import json
from operator import itemgetter
from google.colab import files
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM
#from keras.layers import Dropout
#%pip install pandas==1.4.1       unpickle problem





#upload file from ur google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# or upload file from local drive

In [ ]:
uploaded = files.upload()


Saving hr(tuples).pkl to hr(tuples).pkl


#or upload file manually intp "files" then run this cell V

In [4]:
with open("/content/drive/MyDrive/Colab Notebooks/hr(tuples).pkl", "rb") as f: 
    data = pickle.load(f) 

In [ ]:
#data = pd.read_pickle("/content/hr(tuples).pkl")
%pip install pandas==1.4.1

'1.2.5'

In [5]:
print(data)

      Participant           Stage       0:0.0  0:0.099791  0:0.199582  \
0              22        STIMULUS   (87, 602)   (87, 602)   (87, 602)   
1              22  QUESTIONNAIRES   (85, 921)   (85, 921)   (85, 921)   
2              22         WASHOUT   (80, 749)   (80, 792)   (80, 792)   
3              22        STIMULUS   (81, 719)   (81, 719)   (81, 719)   
4              22  QUESTIONNAIRES   (83, 702)   (82, 702)   (83, 702)   
...           ...             ...         ...         ...         ...   
1339           64        STIMULUS   (78, 516)   (78, 516)   (78, 516)   
1340           64  QUESTIONNAIRES   (77, 799)   (77, 799)   (77, 816)   
1341           64         WASHOUT    (0, 987)    (0, 987)    (0, 987)   
1342           64        STIMULUS   (69, 829)   (68, 829)   (69, 829)   
1343           64  QUESTIONNAIRES  (66, 1245)  (65, 1245)  (65, 1245)   

     0:0.299373  0:0.399164 0:0.49895500000000004  0:0.598746  \
0     (87, 602)   (87, 602)             (87, 602)   (87, 6

In [22]:
def getter(x):
    if isinstance(x,tuple):
        return itemgetter(0)(x)
    else:
        return x

#data = pd.read_pickle("hr(tuples).pkl")
mask = data["Participant"] ==  22
mask2 = data["Stage"] == "STIMULUS"
mask3 = data["Target"]== "BASELINE"
#print(data.loc[ mask2,[*data.columns[2:500],*data.columns[-15:]]].applymap(lambda x:getter(x)))
#print(data.loc[mask2,[*data.columns[2:6292]]].applymap(lambda x:getter(x)))#xtrain->particpants hr values
#print(data.loc[mask2,"Target"])#ytrain-> target emo

print(data.loc[mask2 , "Participant"].unique())


[22 23 24 25 26 27 28 29 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46
 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64]


#splitting by participants

In [84]:
pnumber=data.loc[mask2 , "Participant"].unique()
xtrain=[]
ytrain=[]
xtest=[]
ytest=[]
#30 for training ,rest for test
for i in range (len(pnumber)):
  for j in range(11):
      if i<30:
        xtrain.append(data.loc[(data['Participant'] == pnumber[i]) & (data['Stage'] == 'STIMULUS'),[*data.columns[2:6292]]].applymap(lambda x:getter(x)).iloc[j])
        ytrain.append(data.loc[(data['Participant'] == pnumber[i]) & (data['Stage'] == 'STIMULUS'),"Target"].iloc[j])
      else:
        xtest.append(data.loc[(data['Participant'] == pnumber[i]) & (data['Stage'] == 'STIMULUS'),[*data.columns[2:6292]]].applymap(lambda x:getter(x)).iloc[j])
        ytest.append(data.loc[(data['Participant'] == pnumber[i]) & (data['Stage'] == 'STIMULUS'),"Target"].iloc[j])

In [93]:
xtrain1=np.array(xtrain)
xtest1=np.array(xtest)
ytrain1=np.array(ytrain)
ytest1=np.array(ytest)
print(xtrain1.shape)
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(ytrain1)
y_test = label_encoder.fit_transform(ytest1)
x_train,x_test=np.reshape(xtrain1,(xtrain1.shape[0],xtrain1.shape[1],1)),np.reshape(xtest1,(xtest1.shape[0],xtest1.shape[1],1))
print(x_train.shape)

(330, 6290)
(330, 6290, 1)


#random signal split

In [48]:
xtrain=data.loc[mask2,[*data.columns[2:6292]]].applymap(lambda x:getter(x))
ytrain=data.loc[mask2,"Target"]

In [ ]:
x_train=[]
y_train=[]
label_encoder = LabelEncoder()
x_train,y_train=np.array(xtrain),np.array(ytrain)
y_train = label_encoder.fit_transform(y_train)
label_encoder_name_mapping = dict(zip(label_encoder.classes_,
                                         label_encoder.transform(label_encoder.classes_)))
x_train,  x_test, y_train, y_test = train_test_split(x_train, y_train, train_size = 0.7, random_state = 123)
x_train,x_test=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1)),np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
#print(label_encoder_name_mapping)
print(x_train)

[[[88]
  [88]
  [88]
  ...
  [ 0]
  [ 0]
  [ 0]]

 [[76]
  [76]
  [76]
  ...
  [ 0]
  [ 0]
  [ 0]]

 [[89]
  [89]
  [88]
  ...
  [ 0]
  [ 0]
  [ 0]]

 ...

 [[95]
  [95]
  [95]
  ...
  [ 0]
  [ 0]
  [ 0]]

 [[69]
  [70]
  [69]
  ...
  [ 0]
  [ 0]
  [ 0]]

 [[72]
  [72]
  [72]
  ...
  [ 0]
  [ 0]
  [ 0]]]


In [95]:
#initalizing the RNN
regressor =tf.keras.Sequential()
#adding the first lstm layer and some dropout regularisation
regressor.add(tf.keras.layers.LSTM(units=50,return_sequences=True,input_shape=(x_train.shape[1],1)))
regressor.add(tf.keras.layers.Dropout(0.2))
#adding the 2nd lstm layer and some dropout regularisation
regressor.add(tf.keras.layers.LSTM(units=50,return_sequences=True))
regressor.add(tf.keras.layers.Dropout(0.2))

regressor.add(tf.keras.layers.Flatten())

#adding the output layer
regressor.add (tf.keras.layers.Dense(units=11,activation='softmax'))
#compiling the RNN
regressor.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics='accuracy')
print(regressor.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 6290, 50)          10400     
                                                                 
 dropout (Dropout)           (None, 6290, 50)          0         
                                                                 
 lstm_1 (LSTM)               (None, 6290, 50)          20200     
                                                                 
 dropout_1 (Dropout)         (None, 6290, 50)          0         
                                                                 
 flatten (Flatten)           (None, 314500)            0         
                                                                 
 dense (Dense)               (None, 11)                3459511   
                                                                 
Total params: 3,490,111
Trainable params: 3,490,111
Non-

In [96]:
#Fitting the RNN to the training set
history=regressor.fit(x_train,y_train,validation_split = 0.2,epochs=50,batch_size=36,callbacks=[
    tf.keras.callbacks.EarlyStopping(monitor='val_loss',
    patience=5,
    
    restore_best_weights=True)
])

Epoch 1/50
8/8 [==============================] - 78s 9s/step - loss: 4.4511 - accuracy: 0.3333 - val_loss: 2.5005 - val_accuracy: 0.5606
Epoch 2/50
8/8 [==============================] - 71s 9s/step - loss: 1.8436 - accuracy: 0.5644 - val_loss: 1.6914 - val_accuracy: 0.5909
Epoch 3/50
8/8 [==============================] - 73s 9s/step - loss: 1.2382 - accuracy: 0.6212 - val_loss: 1.0899 - val_accuracy: 0.6970
Epoch 4/50
8/8 [==============================] - 70s 9s/step - loss: 0.9812 - accuracy: 0.7045 - val_loss: 1.0827 - val_accuracy: 0.6818
Epoch 5/50
8/8 [==============================] - 72s 9s/step - loss: 0.9818 - accuracy: 0.6818 - val_loss: 0.9971 - val_accuracy: 0.7121
Epoch 6/50
8/8 [==============================] - 72s 9s/step - loss: 0.7878 - accuracy: 0.6894 - val_loss: 0.8010 - val_accuracy: 0.6515
Epoch 7/50
8/8 [==============================] - 68s 9s/step - loss: 0.6651 - accuracy: 0.7121 - val_loss: 0.7139 - val_accuracy: 0.7121
Epoch 8/50
8/8 [==================

In [97]:
model_acc = regressor.evaluate(x_test, y_test, verbose=0)[1]
print(f"Test Accuracy : {round(model_acc, 3)*100} %")
     

Test Accuracy : 69.69999999999999 %


cross validation part for particpants